# Análise multivariada

## Preparando o ambiente

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import json

## Carregando os dados

In [ ]:
dataset = pd.read_csv('../data/complete_dataset_region/complete_dataset_region.csv')
dataset.head()

## Matriz de correlação

In [ ]:
dataset.corr()

In [ ]:
sns.set(rc={'figure.figsize':(20,15)})
sns.heatmap(dataset.corr(), annot=True, vmin=-1, vmax=1, center=0, mask=np.triu(dataset.corr()))
plt.title('Matriz de correlação das métricas do WHR')

plt.show()


* As correlações faze sentido na realidade, expecialmente no score: `positive_affect` tem correlação positiva enquanto `negative_affect` tem correlação negativa.
* GDP tem uma correlação grande com o score, a segunda maior correlação presente nos dados depois da expectativa de vida. Ambas correlações positivas dentro do esperado.
* Não existe nenhuma correlação extremamente positiva ou negativa, o que evita com que variáveis sejam removidas por causa disso. 
* A variável com menor correlação total é o ano.

## Relação entre o score e o número de pesquisas realizadas pelo país

In [ ]:
score_by_country_cout = dataset.groupby('country').describe()['score'][['count', 'mean']]

sns.displot(data=score_by_country_cout, x="count", y="mean", height=8)
plt.title('Média do score de um país pelo número de vezes que a pesquisa foi realizada')
plt.show()


Não parece apresentar qualquer relação entre esses dois elementos. Existe correlação?

In [ ]:
score_by_country_cout.corr()

A correlação entre o score e o número de vezes que aparece um país é baixa, praticamente irrelevante.

## Relação entre o score e o número de pesquisas realizadas por região

Parte dessa observação ajuda a verificar bias nos dados de percepção de vida para regiões com mais países.

In [ ]:
score_by_region_cout = dataset.groupby('region').describe()['score'][['count', 'mean']]

sns.displot(data=score_by_region_cout, x="count", y="mean", height=8)
plt.title('Média do score de uma região do mundo pelo número de vezes que a pesquisa foi realizada em países dela')
plt.show()


In [ ]:
score_by_region_cout.corr()

Aqui existe uma correlação negativa mais intensa, que pode indicar bias ou também que os países pesquisados mais vezes tendem a ter índices menores.

## Novos países acrescentados vs países ausentes em relação a pesquisa anterior

In [ ]:
first_year = np.min(dataset['year'])
last_year = np.max(dataset['year'])
current_year = first_year + 1
result = []

while(current_year<=last_year):
    countries_before = list(dataset[dataset['year']==current_year-1]['country'].unique())
    countries_current = list(dataset[dataset['year']==current_year]['country'].unique())

    n_added_countries = len(set(countries_current).difference(set(countries_before)))
    n_absent_countries = len(set(countries_before).difference(set(countries_current)))

    result.append({
        'year': current_year,
        'n_added_countries': n_added_countries,
        'n_absent_countries': n_absent_countries
    })

    current_year += 1

result = pd.DataFrame(result)
result


In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.lineplot(data=result, x='year', y='n_added_countries')
sns.lineplot(data=result, x='year', y='n_absent_countries')
plt.title('Número de países adicionados e removidos em cada ano')
plt.xlabel('Ano')
plt.ylabel('Número de países')
plt.show()

Depois dos primeiros anos, a variação dos países é pequena, exceto pelo ano 2020.

## GDP ao longo do tempo

In [ ]:
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='gdp')
plt.title('GDP Mundial ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('GDP')
plt.show()

Com poucas variações, o GDP mundial se manteve estável entre 2010 e 2019. Antes disso eram poucos países e o ano de 2020 também teve menos países na pesquisa.

**Um dos fatores com maior correlação com o score (o GDP) teve um impacto inverso pela número menor de países na pesquisa. Com isso, relações com a pandemia são complicadas de fazer.**

In [ ]:
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='gdp', hue='region', ci=0, legend='brief')
plt.title('GDP Mundial ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('GDP')
plt.show()

Há uma leve tendência geral de crescimento para as regiões abaixo de 10.5. Por curiosidade como está o Brasil comparado à sua região e com o mundo?

In [ ]:
brazil_gdp = dataset[dataset['country']=='brazil'][['year','gdp']]
brazil_gdp['name'] = 'brazil'
lat_america_gdp = dataset[dataset['region']=='latin amer. and carib'].query("country != 'brazil'")[['year','gdp']]
lat_america_gdp['name'] = 'latin america'
brazil_vs_america_gdp = brazil_gdp.append(lat_america_gdp, ignore_index=True)


In [ ]:
sns.lineplot(data=brazil_vs_america_gdp, x='year', y='gdp', hue='name')
plt.title('GPD Brasil ao longo do tempo em comparação com América Latina')
plt.show()

## O papel do social support

1. Relação entre social_support e score


In [ ]:
df = pd.DataFrame(dataset, columns=['social_support', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})
sns.heatmap(df.corr()[['social_support']].sort_values(by=['social_support'], ascending=False), annot=True, vmin=-1, vmax=1, center=0, mask=np.triu(df.corr()[['social_support']].sort_values(by=['social_support'], ascending=False)))

plt.title('Matriz de correlação entre social_support e score')
plt.show()

Há uma tendência de correlação entre os dois atributos. Já que a segurança de ter alguém que irá apoiar em caso de necessidade possa vir ser um fator positivo.

2. Relação entre social_support e região do mundo

In [ ]:
sns.set(rc={"figure.figsize":(15, 10)})
sns.barplot(x="social_support", y="region", data=dataset, ci = None)
plt.title('Região X Suporte ')
plt.ylabel('Regiões')
plt.xlabel('Suporte social')
plt.show()



Mais da metade das regiões acreditam que tem pessoas com que possam contar em caso de necessidade

3. Social_support (total e por região) ao longo do tempo

Mundial

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='social_support')
plt.title('Social support Mundial ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('Social support')
plt.show()

Há pouca ocilação ao longo do ano, tendo rompido a minima apenas em 2008 (possivelmente pela crise de 2008) e 2015.

Por região

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='social_support', hue='region', ci=0, legend='brief')
plt.title('Social support Mundial ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('Social support')
plt.show()

## HLE (healthy life expectancy)

* Relação entre hle e score

In [ ]:
df = pd.DataFrame(dataset, columns=['hle', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})
sns.heatmap(df.corr()[['hle']].sort_values(by=['hle'], ascending=False), annot=True, vmin=-1, vmax=1, center=0, mask=np.triu(df.corr()[['hle']].sort_values(by=['hle'], ascending=False)))

plt.title('Matriz de correlação entre hle e score')
plt.show()

Parece apresentar uma relação positiva entre esses dois elementos indicando a possibilidade de uma expectativa maior de vida aumentar o score

* Relação entre hle e região do mundo


In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.barplot(x="hle", y="region", data=dataset.sort_values(by=['hle'], ascending=True), ci = None)
plt.title('HLE X Suporte ')
plt.ylabel('Regiões')
plt.xlabel('HLE')
plt.show()


Regiões com mais poder aquisitivo parece ter uma vantagem perante os mais sub-desenvolvidos.

* hle (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='hle')

plt.title('Healty life expectancy Mundial ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('Healty life expectancy')
plt.show()

A linha de tendência de alta pode ser representada pelo avanço na medicina, acesso a medicações e exercicios regulares virem a se tornar populares nas classes mais baixas

## FREEDOM

* Relação entre freedom e score

In [ ]:
df = pd.DataFrame(dataset, columns=['freedom', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})
sns.heatmap(df.corr()[['freedom']].sort_values(by=['freedom'], ascending=False), annot=True, vmin=-1, vmax=1, center=0, mask=np.triu(df.corr()[['freedom']].sort_values(by=['freedom'], ascending=False)))

plt.title('Matriz de correlação entre freedom e score')
plt.show()


Liberdade tem um alto impacto no atributo positive_affect, o que pode fazer com que o score baixe

* Relação entre freedom e região do mundo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.barplot(x="freedom", y="region", data=dataset, ci = None)
plt.title('Regiões X Liberdade ')
plt.ylabel('Regiões')
plt.xlabel('Nível de liberdade')
plt.show()


O nivel de liberdade independentemente da região, tem um valor satisfatório, sendo apenas regiões consideradas "mais democraticas" rompendo a pontuação de 0.8

* freedom (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='freedom')

plt.title('Liberdade ao longo do tempo (2006-2021)')
plt.xlabel('Ano')
plt.ylabel('Nível de liberdade')
plt.show()

Parece que a a pontuação de liberdade cai quando há crises econômicas como em 2008 e 2012. Além de 2021 com a pandemia

## Generosity

* Relação entre generosity e score

In [ ]:
df = pd.DataFrame(dataset, columns=['generosity', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})

sns.heatmap(
  df.corr()[['generosity']].sort_values(
    by=['generosity'],
    ascending=False
  ),
  annot=True,
  vmin=-1,
  vmax=1,
  center=0,
  mask=np.triu(df.corr()[['generosity']].sort_values(
    by=['generosity'],
    ascending=False
    )
  )
)

plt.title('Matriz de correlação entre generosidade e score')
plt.show()

Aparentemente não há relação entre os atributos de `generosoty` e `score`

* Relação entre generosity e região do mundo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.barplot(x="generosity", y="region", data=dataset, ci = None)

plt.title('Regiões X Generosidade ')

plt.ylabel('Regiões')
plt.xlabel('Generosidade')

plt.show()

Em termos de generosidade menos da metade tem o costume de doar a instituições de caridade

* generosity (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='generosity')

plt.title('Generosidade ao longo do tempo (2006-2021)')

plt.xlabel('Ano')
plt.ylabel('Generosidade')

plt.show()

Sendo um atributo que nunca teve altos indices, mesmo com o ano de 2015 batendo na linha de tendencia de alta o indice praticamente baixou ao longo dos anos

## Corruption

* Relação entre corruption e score

In [ ]:
df = pd.DataFrame(dataset, columns=['corruption', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})

sns.heatmap(
  df.corr()[['corruption']].sort_values(
    by=['corruption'],
    ascending=False
  ),
  annot=True,
  vmin=-1,
  vmax=1,
  center=0,
  mask=np.triu(df.corr()[['corruption']].sort_values(
    by=['corruption'],
    ascending=False
    )
  )
)

plt.title('Matriz de correlação entre generosidade e score')
plt.show()

Aparentemente não há relação entre os atributos de `corruption` e `score`

* Relação entre corruption e região do mundo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.barplot(
  x="corruption",
  y="region",
  data=dataset,
  ci = None,
  # TODO: order
)

plt.title('Regiões X Corrupção ')

plt.ylabel('Regiões')
plt.xlabel('Corrupção')

plt.show()

Indices altos em praticamente todas as regiões, tendo apenas a oceania abaixo de 0.5, talvez pela quantidade de países o indice seja prejudicado diante dos demais

* corruption (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='corruption')

plt.title('Corrupção ao longo do tempo (2006-2021)')

plt.xlabel('Ano')
plt.ylabel('Corrupção')

plt.show()

Não faz sentindo esse atributo cair com o passar do tempo, uma explicação seria um desentendimento da população perante aos seus governantes

## Positive affect

* Relação entre positive_affect e score

In [ ]:
df = pd.DataFrame(dataset, columns=['positive_affect', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})

sns.heatmap(
  df.corr()[['positive_affect']].sort_values(
    by=['positive_affect'],
    ascending=False
  ),
  annot=True,
  vmin=-1,
  vmax=1,
  center=0,
  mask=np.triu(df.corr()[['positive_affect']].sort_values(
    by=['positive_affect'],
    ascending=False
    )
  )
)

plt.title('Matriz de correlação entre afeto positivo e score')
plt.show()

A correlação entre esse dois atributos é positiva mas fraca

* Relação entre positive_affect e região do mundo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
region_affect = pd.DataFrame(dataset.groupby('region').describe()['positive_affect']['mean']).reset_index().sort_values(by='mean', ascending=False)
sns.barplot(data=region_affect, x='region', y='mean')
plt.show()

In [ ]:
sorted_values = dataset.sort_values('positive_affect')['positive_affect']

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})
#region_affect = pd.DataFrame(dataset.groupby('region').describe()['positive_affect']['mean']).reset_index()
sns.barplot(data=dataset, x='region', y='positive_affect')
plt.show()

* positive_affect (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.barplot(
  x="positive_affect",
  y="region",
  data=dataset,
  ci = None,
)

plt.title('Regiões X Afeto positivo')

plt.ylabel('Regiões')
plt.xlabel('Afeto positivo')

plt.show()

Em sua grande maioria é positivo o indicador, nenhum sendo abaixo dos 0.5

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='positive_affect')

plt.title('Afeto positivo ao longo do tempo (2006-2021)')

plt.xlabel('Ano')
plt.ylabel('Afeto')

plt.show()

Pode se dizer que apesar das pequenas flutuações, o indice se manteve ao longo do tempo

* verificar score dos países com positive_affect = 0

In [ ]:
there_are_zero_values = dataset.query('positive_affect == 0')
there_are_zero_values

Não a países com o atributo 0

## Negative affect

* Relação entre negative_affect e score

In [ ]:
df = pd.DataFrame(dataset, columns=['negative_affect', 'score'])

sns.set(rc={"figure.figsize":(5, 9)})

sns.heatmap(
  df.corr()[['negative_affect']].sort_values(
    by=['negative_affect'],
    ascending=False
  ),
  annot=True,
  vmin=-1,
  vmax=1,
  center=0,
  mask=np.triu(df.corr()[['negative_affect']].sort_values(
    by=['negative_affect'],
    ascending=False
    )
  )
)

plt.title('Matriz de correlação entre afeto positivo e score')
plt.show()

Aparentemente não há relação entre os dois atributos

* Relação entre negative_affect e região do mundo

* negative_affect (total e por região) ao longo do tempo

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.barplot(
  x="negative_affect",
  y="region",
  data=dataset.sort_values(by=['negative_affect'], ascending=[False]),
  ci = None
)

plt.title('Regiões X Afeto negativo')

plt.ylabel('Regiões')
plt.xlabel('Afeto negativo')

plt.show()

Em contra proposta de afeto positivo, o afeto negativo tem indices baixos. O que é bem relevante.

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='negative_affect')

plt.title('Afeto negativo ao longo do tempo (2006-2021)')

plt.xlabel('Ano')
plt.ylabel('Afeto')

plt.show()

Mostra uma linha de tendencia de alta constante, podendo ser um indice preocupante

* verificar score dos países com negative_affect = 0 e analisar se são os mesmos com positive_affect = 0

In [ ]:
there_are_zero_values = dataset['negative_affect'].min()
there_are_zero_values

Não há atributo com o valor 0

## Year

* quais países estão presentes em 2005. Eles estão presentes em pesquisas posteriores? Caso não talvez devam ser descartados


In [ ]:
present_in_2005 = dataset.query('year == 2005')

qtd_country_in_2005 = len(present_in_2005)

present_in_2005

Sim, são países presentes em pesquisas posteriores.

* Quantos países novos entram e quantos são ignorados nas pesquisas do ano seguinte?

In [ ]:
present_in_2006 = dataset.query('year == 2006')
qtd_country_in_2006 = len(present_in_2006)

present_in_2006

Quantidade de novos países nas pesquisas

In [ ]:
country_count_in_2005 = "{count} presentes em 2005".format(count = qtd_country_in_2005)
country_count_in_2006 = "{count} em 2006".format(count = qtd_country_in_2006)
new_country_count = "{count} novos países.".format(count = qtd_country_in_2006 - qtd_country_in_2005)

result = new_country_count + ' Sendo ' + country_count_in_2005 + ' e ' + country_count_in_2006

result

Países de 2005 encontrados no ano de 2006

In [ ]:
mergedCountries = pd.merge(present_in_2006, present_in_2005, on=['country'], how='inner')
mergedCountries

* Presença das regiões nas pesquisas ao longo do ano?

In [ ]:
sns.relplot(
  data=dataset,
  x="year",
  y="region",
  height=10,
  aspect=2,
  hue="year",
  palette=sns.diverging_palette(250, 30, l=65, center="dark", as_cmap=True)
)

plt.title('Participações ao longo do tempo')

plt.ylabel('Regiões representantes')
plt.xlabel('Ano de participação')

plt.show()

Aparentemente houveram regiões não participantes apenas no início das pesquisas

* Como é a representação das regiões do mundo ao longo do tempo?

In [ ]:

x = dataset['year']
y = dataset['region']
z = dataset.groupby('country').describe()['score'][['count']]['count']

plt.scatter(x, y, s=z*50, alpha=0.5)

plt.title('Densidade de países participantes ao longo do tempo')

plt.ylabel('Regiões representantes')
plt.xlabel('Ano de participação')

plt.show()


* Qual o score médio ao longo do tempo?

In [ ]:
sns.set(rc={"figure.figsize":(20, 10)})

sns.lineplot(data=dataset[dataset['year']>2005], x='year', y='score')

plt.title('Score ao longo do tempo (2006-2021)')

plt.xlabel('Ano')
plt.ylabel('Score')

plt.show()


O score se manteve na mesma escala por um bom tempo, e o que provavelmente baixou novamente foi a pandemia